In [3]:
import numpy as np
import os
import torch
from torch.optim import Adam
from torchvision import models, datasets, transforms
from network import Autoencoder, Conv_Autoencoder
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader


In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

#Fashion NIST data has form 1x1x28x28 (grayscale)
for images, labels in train_loader:
    print(images.shape)  # Output: torch.Size([64, 1, 28, 28])
    print(labels.shape)  # Output: torch.Size([64])
    break  # Just to show one batch

torch.Size([64, 1, 28, 28])
torch.Size([64])
